In [1]:
import numpy as np
import pandas as pd
import pickle

import requests
import time
import sys
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary

In [2]:
"""
    Scope:
        Predict the price per lb of coffee by country given the following set of features will be collected as many years back as available:
        
        Many of these features can be taken from World Bank at:
            https://datacatalog.worldbank.org/search?sort_by=field_wbddh_modified_date&sort_order=DESC&search_api_views_fulltext_op=AND&f[0]=field_wbddh_data_type%3A293
            
        Am working on finding better datasets and sites for the ones I currently don't have (mainly tea and precip datasets)
"""

"\n    Scope:\n        Predict the price per lb of coffee by country given the following set of features will be collected as many years back as available:\n        \n        Many of these features can be taken from World Bank at:\n            https://datacatalog.worldbank.org/search?sort_by=field_wbddh_modified_date&sort_order=DESC&search_api_views_fulltext_op=AND&f[0]=field_wbddh_data_type%3A293\n            \n        Am working on finding better datasets and sites for the ones I currently don't have (mainly tea and precip datasets)\n"

In [3]:
""" 
    Features (by country, per year):
        hours worked per year        hours_worked_df        2000 - 2018
        income level*                income_df              
        age distribution*            age_df                 1960 - 2018
        working population           unemployment_df        1991 - 2017
        education level              edu_df                 1880 - 2018
        tea consumption
        annual precipitation
""" 

' \n    Features (by country, per year):\n        hours worked per year        hours_worked_df        2000 - 2018\n        income level*                income_df              \n        age distribution*            age_df                 1960 - 2018\n        working population           unemployment_df        1991 - 2017\n        education level              edu_df                 1880 - 2018\n        tea consumption\n        annual precipitation\n'

In [4]:
hours_worked_df = pd.read_csv('/Users/joelporcaro/Metis/project2/project2_data/hours_worked.csv')
hours_worked_df.head(10)

,COUNTRY,Country,EMPSTAT,Employment status,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,TE,Total employment,A,Annual,2000,2000,HOUR,Hours,0,Units,NaN,NaN,1779.63,NaN,NaN
1,AUS,Australia,TE,Total employment,A,Annual,2001,2001,HOUR,Hours,0,Units,NaN,NaN,1746.37,NaN,NaN
2,AUS,Australia,TE,Total employment,A,Annual,2002,2002,HOUR,Hours,0,Units,NaN,NaN,1741.82,NaN,NaN
3,AUS,Australia,TE,Total employment,A,Annual,2003,2003,HOUR,Hours,0,Units,NaN,NaN,1735.10,NaN,NaN
4,AUS,Australia,TE,Total employment,A,Annual,2004,2004,HOUR,Hours,0,Units,NaN,NaN,1733.74,NaN,NaN
5,AUS,Australia,TE,Total employment,A,Annual,2005,2005,HOUR,Hours,0,Units,NaN,NaN,1732.26,NaN,NaN
6,AUS,Australia,TE,Total employment,A,Annual,2006,2006,HOUR,Hours,0,Units,NaN,NaN,1721.66,NaN,NaN
7,AUS,Australia,TE,Total employment,A,Annual,2007,2007,HOUR,Hours,0,Units,NaN,NaN,1722.64,NaN,NaN
8,AUS,Australia,TE,Total employment,A,Annual,2008,2008,HOUR,Hours,0,Units,NaN,NaN,1720.88,NaN,NaN
9,AUS,Australia,TE,Total employment,A,Annual,2009,2009,HOUR,Hours,0,Units,NaN,NaN,1692.33,NaN,NaN


In [5]:
hours_worked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 17 columns):
COUNTRY                  1309 non-null object
Country                  1309 non-null object
EMPSTAT                  1309 non-null object
Employment status        1309 non-null object
FREQUENCY                1309 non-null object
Frequency                1309 non-null object
TIME                     1309 non-null int64
Time                     1309 non-null int64
Unit Code                1309 non-null object
Unit                     1309 non-null object
PowerCode Code           1309 non-null int64
PowerCode                1309 non-null object
Reference Period Code    0 non-null float64
Reference Period         0 non-null float64
Value                    1309 non-null float64
Flag Codes               0 non-null float64
Flags                    0 non-null float64
dtypes: float64(5), int64(3), object(9)
memory usage: 173.9+ KB


In [6]:
hours_worked_df.columns

Index(['COUNTRY', 'Country', 'EMPSTAT', 'Employment status', 'FREQUENCY',
       'Frequency', 'TIME', 'Time', 'Unit Code', 'Unit', 'PowerCode Code',
       'PowerCode', 'Reference Period Code', 'Reference Period', 'Value',
       'Flag Codes', 'Flags'],
      dtype='object')

In [7]:
hours_worked_df = hours_worked_df[['Country', 'Time', 'Value']]
hours_worked_df.head()

,Country,Time,Value
0,Australia,2000,1779.63
1,Australia,2001,1746.37
2,Australia,2002,1741.82
3,Australia,2003,1735.10
4,Australia,2004,1733.74


In [8]:
unemployment_df = pd.read_csv('/Users/joelporcaro/Metis/project2/project2_data/unemployment-rate.csv')
unemployment_df.head(10)

,Entity,Code,Year,"Unemployment, total (% of total labor force) (modeled ILO estimate) (% of total labor force)"
0,Afghanistan,AFG,1991,12.031
1,Afghanistan,AFG,1992,10.645
2,Afghanistan,AFG,1993,12.368
3,Afghanistan,AFG,1994,12.299
4,Afghanistan,AFG,1995,8.702
5,Afghanistan,AFG,1996,11.138
6,Afghanistan,AFG,1997,11.109
7,Afghanistan,AFG,1998,11.013
8,Afghanistan,AFG,1999,11.069
9,Afghanistan,AFG,2000,11.184


In [9]:
unemployment_df.columns

Index(['Entity', 'Code', 'Year',
       'Unemployment, total (% of total labor force) (modeled ILO estimate) (% of total labor force)'],
      dtype='object')

In [10]:
unemployment_df = unemployment_df[['Entity', 'Year', 'Unemployment, total (% of total labor force) (modeled ILO estimate) (% of total labor force)']]
unemployment_df = unemployment_df.rename(columns={'Entity': 'Country', 'Unemployment, total (% of total labor force) (modeled ILO estimate) (% of total labor force)': 'Unemployment (%)'})
unemployment_df.head()

,Country,Year,Unemployment (%)
0,Afghanistan,1991,12.031
1,Afghanistan,1992,10.645
2,Afghanistan,1993,12.368
3,Afghanistan,1994,12.299
4,Afghanistan,1995,8.702


In [11]:
max(unemployment_df.Year)

2017

In [12]:
edu_df = pd.read_csv('/Users/joelporcaro/Metis/project2/project2_data/years_shcooling.csv')
edu_df.head(10)

,Entity,Code,Year,Average total years of schooling for adult population (years)
0,Afghanistan,AFG,1870,0.01
1,Afghanistan,AFG,1875,0.01
2,Afghanistan,AFG,1880,0.01
3,Afghanistan,AFG,1885,0.01
4,Afghanistan,AFG,1890,0.01
5,Afghanistan,AFG,1895,0.01
6,Afghanistan,AFG,1900,0.01
7,Afghanistan,AFG,1905,0.01
8,Afghanistan,AFG,1910,0.01
9,Afghanistan,AFG,1915,0.02


In [13]:
edu_df = edu_df.drop(['Code'], axis=1)
edu_df.head()

,Entity,Year,Average total years of schooling for adult population (years)
0,Afghanistan,1870,0.01
1,Afghanistan,1875,0.01
2,Afghanistan,1880,0.01
3,Afghanistan,1885,0.01
4,Afghanistan,1890,0.01


In [14]:
edu_df = edu_df.rename(columns={'Average total years of schooling for adult population (years)': 'Ave Years of Schooling'})
edu_df.head(100)

,Entity,Year,Ave Years of Schooling
0,Afghanistan,1870,0.01
1,Afghanistan,1875,0.01
2,Afghanistan,1880,0.01
3,Afghanistan,1885,0.01
4,Afghanistan,1890,0.01
5,Afghanistan,1895,0.01
6,Afghanistan,1900,0.01
7,Afghanistan,1905,0.01
8,Afghanistan,1910,0.01
9,Afghanistan,1915,0.02


In [15]:
income_path = '/Users/joelporcaro/Metis/project2/project2_data/annual_income/API_NY.ADJ.NNTY.PC.CD_DS2_en_csv_v2_41570.csv'
income_df = pd.read_csv(income_path, header=2)
# income_df['1960'].dropna()

In [16]:
income_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', 'Unnamed: 63'],
      dtype='object')

In [17]:
income_df = income_df.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 63'], axis=1)
income_df.head()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21289.646597,19741.149803,19960.521208,21038.151122,20841.489471,21891.503904,21961.098811,21368.804889,21385.451665,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,385.894399,477.307009,517.448448,554.779215,550.956701,529.876402,502.276410,465.841583,466.079081,NaN
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,735.380805,1108.338717,1494.240377,1789.733615,2113.866121,2498.701951,2341.460543,1949.114680,2148.272969,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3622.330148,3530.690656,3853.165353,3644.843103,3937.277061,4038.014865,3560.787907,3776.111694,4105.150515,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
income_df = income_df.rename(columns={'Country Name': 'Country'})
income_df

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21289.646597,19741.149803,19960.521208,21038.151122,20841.489471,21891.503904,21961.098811,21368.804889,21385.451665,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,385.894399,477.307009,517.448448,554.779215,550.956701,529.876402,502.276410,465.841583,466.079081,NaN
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,735.380805,1108.338717,1494.240377,1789.733615,2113.866121,2498.701951,2341.460543,1949.114680,2148.272969,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3622.330148,3530.690656,3853.165353,3644.843103,3937.277061,4038.014865,3560.787907,3776.111694,4105.150515,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4223.658162,4792.279065,5416.623552,5905.729634,5951.002430,6035.729500,5445.677217,5294.685363,5247.986414,NaN
6,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27355.863877,30537.816942,34564.778241,36057.066577,38070.070199,39332.031393,35654.474779,35465.405184,37374.460621,NaN
7,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6956.115650,8810.359213,11008.524592,11250.021382,11255.021381,10607.406458,12003.351157,11006.029306,12365.901299,NaN
8,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2723.946831,2918.101653,3138.262966,3274.122578,3509.206422,3587.530469,3181.176422,3084.605658,3412.080148,NaN
9,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
age_path = '/Users/joelporcaro/Metis/project2/project2_data/ages/API_SP.POP.1564.TO.ZS_DS2_en_csv_v2_41132.csv'
age_df = pd.read_csv(age_path, header=2)
age_df.head(10)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,53.669919,54.056784,54.383281,54.710292,55.119933,55.631102,...,68.838092,68.882835,68.976476,69.090099,69.165181,69.159774,69.152468,69.003648,68.762184,NaN
1,Afghanistan,AFG,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,55.064659,54.747313,54.579029,54.500882,54.443084,54.376483,...,49.898603,50.319337,50.883182,51.545931,52.253100,52.974876,53.598917,54.174914,54.742297,NaN
2,Angola,AGO,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,52.719671,52.232669,51.635484,51.057092,50.703894,50.660431,...,50.370243,50.327156,50.349958,50.416077,50.505664,50.613384,50.690863,50.787584,50.917618,NaN
3,Albania,ALB,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,54.434567,54.068450,53.695003,53.353190,53.111451,53.010165,...,66.860680,67.520815,68.110597,68.645567,69.103098,69.436421,69.443689,69.415774,69.297902,NaN
4,Andorra,AND,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,ARB,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,53.171002,52.772366,52.421896,52.131270,51.923473,51.807219,...,62.260567,62.395755,62.452398,62.456183,62.451055,62.466139,62.436158,62.477525,62.556051,NaN
6,United Arab Emirates,ARE,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,51.880519,53.279645,54.808308,56.342481,57.804503,59.179181,...,85.872402,85.640650,85.525355,85.437623,85.322682,85.191607,85.028100,84.960562,84.903583,NaN
7,Argentina,ARG,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,63.695389,63.618909,63.572807,63.556931,63.570981,63.611345,...,63.721484,63.796777,63.827041,63.831450,63.843150,63.877761,63.880519,63.916238,63.973486,NaN
8,Armenia,ARM,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,54.997119,54.116450,53.462856,53.023164,52.767648,52.693961,...,69.545663,69.706505,69.722289,69.611188,69.436416,69.246199,69.015521,68.760873,68.490044,NaN
9,American Samoa,ASM,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
age_df = age_df.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 63'], axis=1)
age_df.head()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,53.669919,54.056784,54.383281,54.710292,55.119933,55.631102,56.075544,56.703126,57.414449,...,69.161089,68.838092,68.882835,68.976476,69.090099,69.165181,69.159774,69.152468,69.003648,68.762184
1,Afghanistan,55.064659,54.747313,54.579029,54.500882,54.443084,54.376483,54.033502,53.738502,53.489183,...,49.815480,49.898603,50.319337,50.883182,51.545931,52.253100,52.974876,53.598917,54.174914,54.742297
2,Angola,52.719671,52.232669,51.635484,51.057092,50.703894,50.660431,50.358904,50.388691,50.631082,...,50.317153,50.370243,50.327156,50.349958,50.416077,50.505664,50.613384,50.690863,50.787584,50.917618
3,Albania,54.434567,54.068450,53.695003,53.353190,53.111451,53.010165,52.954356,53.062977,53.293096,...,66.619749,66.860680,67.520815,68.110597,68.645567,69.103098,69.436421,69.443689,69.415774,69.297902
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
age_df = age_df.rename(columns={'Country Name': 'Country'})
age_df.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,53.669919,54.056784,54.383281,54.710292,55.119933,55.631102,56.075544,56.703126,57.414449,...,69.161089,68.838092,68.882835,68.976476,69.090099,69.165181,69.159774,69.152468,69.003648,68.762184
1,Afghanistan,55.064659,54.747313,54.579029,54.500882,54.443084,54.376483,54.033502,53.738502,53.489183,...,49.815480,49.898603,50.319337,50.883182,51.545931,52.253100,52.974876,53.598917,54.174914,54.742297
2,Angola,52.719671,52.232669,51.635484,51.057092,50.703894,50.660431,50.358904,50.388691,50.631082,...,50.317153,50.370243,50.327156,50.349958,50.416077,50.505664,50.613384,50.690863,50.787584,50.917618
3,Albania,54.434567,54.068450,53.695003,53.353190,53.111451,53.010165,52.954356,53.062977,53.293096,...,66.619749,66.860680,67.520815,68.110597,68.645567,69.103098,69.436421,69.443689,69.415774,69.297902
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
income_df.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21289.646597,19741.149803,19960.521208,21038.151122,20841.489471,21891.503904,21961.098811,21368.804889,21385.451665,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,385.894399,477.307009,517.448448,554.779215,550.956701,529.876402,502.276410,465.841583,466.079081,NaN
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,735.380805,1108.338717,1494.240377,1789.733615,2113.866121,2498.701951,2341.460543,1949.114680,2148.272969,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3622.330148,3530.690656,3853.165353,3644.843103,3937.277061,4038.014865,3560.787907,3776.111694,4105.150515,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# income_df.columns
# income_df.info()

years_to_drop = [str(year) for year in range(1960,2000)]
income_df = income_df.drop(years_to_drop, axis=1)

In [24]:
income_col_trans = ['2000', '2001', '2002','2003', '2004', '2005', '2006',
                    '2007', '2008', '2009','2010', '2011', '2012', '2013',
                    '2014', '2015', '2016', '2017', '2018']

In [25]:
income_df

,Country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,18248.936391,17924.265302,16894.476620,18140.298176,19503.280285,16199.594853,20567.526976,19423.428955,23229.613614,21289.646597,19741.149803,19960.521208,21038.151122,20841.489471,21891.503904,21961.098811,21368.804889,21385.451665,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,385.894399,477.307009,517.448448,554.779215,550.956701,529.876402,502.276410,465.841583,466.079081,NaN
2,Angola,4.330681,109.244335,357.799021,437.003981,440.752798,563.934596,856.912210,819.694989,367.194821,735.380805,1108.338717,1494.240377,1789.733615,2113.866121,2498.701951,2341.460543,1949.114680,2148.272969,NaN
3,Albania,1074.702633,1223.107257,1343.238986,1743.114101,2232.469483,2500.857583,2808.973837,3333.996140,3961.899406,3622.330148,3530.690656,3853.165353,3644.843103,3937.277061,4038.014865,3560.787907,3776.111694,4105.150515,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,2078.525199,2061.574139,2060.358257,2221.165421,2513.896490,2925.217915,3402.880771,3956.793176,4790.969646,4223.658162,4792.279065,5416.623552,5905.729634,5951.002430,6035.729500,5445.677217,5294.685363,5247.986414,NaN
6,United Arab Emirates,30680.293763,28940.377845,28249.248256,29500.425897,32307.054290,33402.255838,35940.371519,36627.751525,37339.636199,27355.863877,30537.816942,34564.778241,36057.066577,38070.070199,39332.031393,35654.474779,35465.405184,37374.460621,NaN
7,Argentina,6542.999691,6089.740055,1977.682054,2675.967837,3176.814458,3794.102817,4893.936048,6111.502302,7607.623017,6956.115650,8810.359213,11008.524592,11250.021382,11255.021381,10607.406458,12003.351157,11006.029306,12365.901299,NaN
8,Armenia,539.692801,606.000764,682.830784,815.858696,1047.871900,1475.613727,1959.746874,2886.893681,3742.407790,2723.946831,2918.101653,3138.262966,3274.122578,3509.206422,3587.530469,3181.176422,3084.605658,3412.080148,NaN
9,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# income stuff here

In [27]:
income_df

,Country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,18248.936391,17924.265302,16894.476620,18140.298176,19503.280285,16199.594853,20567.526976,19423.428955,23229.613614,21289.646597,19741.149803,19960.521208,21038.151122,20841.489471,21891.503904,21961.098811,21368.804889,21385.451665,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,385.894399,477.307009,517.448448,554.779215,550.956701,529.876402,502.276410,465.841583,466.079081,NaN
2,Angola,4.330681,109.244335,357.799021,437.003981,440.752798,563.934596,856.912210,819.694989,367.194821,735.380805,1108.338717,1494.240377,1789.733615,2113.866121,2498.701951,2341.460543,1949.114680,2148.272969,NaN
3,Albania,1074.702633,1223.107257,1343.238986,1743.114101,2232.469483,2500.857583,2808.973837,3333.996140,3961.899406,3622.330148,3530.690656,3853.165353,3644.843103,3937.277061,4038.014865,3560.787907,3776.111694,4105.150515,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,2078.525199,2061.574139,2060.358257,2221.165421,2513.896490,2925.217915,3402.880771,3956.793176,4790.969646,4223.658162,4792.279065,5416.623552,5905.729634,5951.002430,6035.729500,5445.677217,5294.685363,5247.986414,NaN
6,United Arab Emirates,30680.293763,28940.377845,28249.248256,29500.425897,32307.054290,33402.255838,35940.371519,36627.751525,37339.636199,27355.863877,30537.816942,34564.778241,36057.066577,38070.070199,39332.031393,35654.474779,35465.405184,37374.460621,NaN
7,Argentina,6542.999691,6089.740055,1977.682054,2675.967837,3176.814458,3794.102817,4893.936048,6111.502302,7607.623017,6956.115650,8810.359213,11008.524592,11250.021382,11255.021381,10607.406458,12003.351157,11006.029306,12365.901299,NaN
8,Armenia,539.692801,606.000764,682.830784,815.858696,1047.871900,1475.613727,1959.746874,2886.893681,3742.407790,2723.946831,2918.101653,3138.262966,3274.122578,3509.206422,3587.530469,3181.176422,3084.605658,3412.080148,NaN
9,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
income_first_country = income_df[0:1]
income_first_country.transpose()
# income_trans_test = income_first2_countries[income_col_trans]
# income_trans_test.T


# income_trans_test.transpose()

,0
Country,Aruba
2000,18248.9
2001,17924.3
2002,16894.5
2003,18140.3
2004,19503.3
2005,16199.6
2006,20567.5
2007,19423.4
2008,23229.6


In [29]:
for i in income_df.iloc[0,0]:
    print(i)

A
r
u
b
a


In [30]:
# arubas = income_df.loc[Country='Aruba']
income_df

,Country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,18248.936391,17924.265302,16894.476620,18140.298176,19503.280285,16199.594853,20567.526976,19423.428955,23229.613614,21289.646597,19741.149803,19960.521208,21038.151122,20841.489471,21891.503904,21961.098811,21368.804889,21385.451665,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,385.894399,477.307009,517.448448,554.779215,550.956701,529.876402,502.276410,465.841583,466.079081,NaN
2,Angola,4.330681,109.244335,357.799021,437.003981,440.752798,563.934596,856.912210,819.694989,367.194821,735.380805,1108.338717,1494.240377,1789.733615,2113.866121,2498.701951,2341.460543,1949.114680,2148.272969,NaN
3,Albania,1074.702633,1223.107257,1343.238986,1743.114101,2232.469483,2500.857583,2808.973837,3333.996140,3961.899406,3622.330148,3530.690656,3853.165353,3644.843103,3937.277061,4038.014865,3560.787907,3776.111694,4105.150515,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,2078.525199,2061.574139,2060.358257,2221.165421,2513.896490,2925.217915,3402.880771,3956.793176,4790.969646,4223.658162,4792.279065,5416.623552,5905.729634,5951.002430,6035.729500,5445.677217,5294.685363,5247.986414,NaN
6,United Arab Emirates,30680.293763,28940.377845,28249.248256,29500.425897,32307.054290,33402.255838,35940.371519,36627.751525,37339.636199,27355.863877,30537.816942,34564.778241,36057.066577,38070.070199,39332.031393,35654.474779,35465.405184,37374.460621,NaN
7,Argentina,6542.999691,6089.740055,1977.682054,2675.967837,3176.814458,3794.102817,4893.936048,6111.502302,7607.623017,6956.115650,8810.359213,11008.524592,11250.021382,11255.021381,10607.406458,12003.351157,11006.029306,12365.901299,NaN
8,Armenia,539.692801,606.000764,682.830784,815.858696,1047.871900,1475.613727,1959.746874,2886.893681,3742.407790,2723.946831,2918.101653,3138.262966,3274.122578,3509.206422,3587.530469,3181.176422,3084.605658,3412.080148,NaN
9,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:

# aruba

In [32]:
# aruba = income_df.iloc[0,0]
aruba = income_df.Country[0]       # selecting the country
aruba = pd.Series(aruba)           # converting to pandas series
aruba = aruba.repeat(18)         # repeating process
aruba = aruba.reset_index()      # must reset index for concat to work
aruba = aruba.loc[:,0]           # selects the cleaner version of the series

yearsAruba = income_df.columns[2:]
yearsAruba = pd.Series(yearsAruba)  # creates years. probably only need to do this once and not in the loop
z = pd.concat([aruba, yearsAruba], axis=1)  # concatenating aruba with the years
# # z = z.drop(['index'], axis=1)

values = income_df.loc[0].values[2:]   # selecting the values for aruba

z = pd.concat([z, pd.Series(values)], axis=1) # concatenating aruba/years with the values 
z
# values

,0,0,0
0,Aruba,2001,17924.3
1,Aruba,2002,16894.5
2,Aruba,2003,18140.3
3,Aruba,2004,19503.3
4,Aruba,2005,16199.6
5,Aruba,2006,20567.5
6,Aruba,2007,19423.4
7,Aruba,2008,23229.6
8,Aruba,2009,21289.6
9,Aruba,2010,19741.1


In [33]:
income_df.columns

Index(['Country', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018'],
      dtype='object')

In [34]:
# Afghanistan = income_df.iloc[0,0]
Afghanistan = income_df.Country[1]       # selecting the country
Afghanistan = pd.Series(Afghanistan)           # converting to pandas series
Afghanistan = Afghanistan.repeat(18)         # repeating process
Afghanistan = Afghanistan.reset_index()      # must reset index for concat to work
Afghanistan = Afghanistan.loc[:,0]           # selects the cleaner version of the series

yearsAfghanistan = income_df.columns[2:]
yearsAfghanistan = pd.Series(yearsAfghanistan)  # creates years. probably only need to do this once and not in the loop
z2 = pd.concat([Afghanistan, yearsAfghanistan], axis=1)  # concatenating aruba with the years
# # z = z.drop(['index'], axis=1)

values2 = income_df.loc[1].values[2:]   # selecting the values for aruba

z2 = pd.concat([z2, pd.Series(values2)], axis=1) # concatenating aruba/years with the values 
z2

Afghanistan




0     Afghanistan
1     Afghanistan
2     Afghanistan
3     Afghanistan
4     Afghanistan
5     Afghanistan
6     Afghanistan
7     Afghanistan
8     Afghanistan
9     Afghanistan
10    Afghanistan
11    Afghanistan
12    Afghanistan
13    Afghanistan
14    Afghanistan
15    Afghanistan
16    Afghanistan
17    Afghanistan
Name: 0, dtype: object

In [35]:
z3 = pd.concat([z, z2])
z3 = z3.reset_index(drop=True)
z3

,0,0,0
0,Aruba,2001,17924.3
1,Aruba,2002,16894.5
2,Aruba,2003,18140.3
3,Aruba,2004,19503.3
4,Aruba,2005,16199.6
5,Aruba,2006,20567.5
6,Aruba,2007,19423.4
7,Aruba,2008,23229.6
8,Aruba,2009,21289.6
9,Aruba,2010,19741.1


In [58]:
income_years = pd.Series(income_df.columns[2:])
new_income_df = pd.DataFrame()
country_count = 0

for country in income_df.Country[:]:
    
    # creating country series
    current_country = pd.Series(country)
    current_country = current_country.repeat(18).reset_index().loc[:,0]
    
    # adding the years
    country_plus_years = pd.concat([current_country, income_years], axis=1)
    
    # value selection
    current_country_values = income_df.loc[country_count].values[2:]
    
    # putting it all together 
    values_series = pd.concat([country_plus_years, pd.Series(current_country_values)], axis=1)
    
    new_income_df = pd.concat([new_income_df, values_series])
    new_income_df = new_income_df.reset_index(drop=True)
    
    country_count += 1
    
new_income_df

,0,0,0
0,Aruba,2001,17924.3
1,Aruba,2002,16894.5
2,Aruba,2003,18140.3
3,Aruba,2004,19503.3
4,Aruba,2005,16199.6
5,Aruba,2006,20567.5
6,Aruba,2007,19423.4
7,Aruba,2008,23229.6
8,Aruba,2009,21289.6
9,Aruba,2010,19741.1


In [37]:
new_income_df.columns = ['Country', 'Year', 'Mean Annual Income ($)']
new_income_df

,Country,Year,Mean Annual Income ($)
0,Aruba,2001,17924.3
1,Aruba,2002,16894.5
2,Aruba,2003,18140.3
3,Aruba,2004,19503.3
4,Aruba,2005,16199.6
5,Aruba,2006,20567.5
6,Aruba,2007,19423.4
7,Aruba,2008,23229.6
8,Aruba,2009,21289.6
9,Aruba,2010,19741.1


In [38]:
# age stuff here

In [39]:
new_income_df = pd.Series()
new_income_df

Series([], dtype: float64)

In [40]:
hours_worked_df = hours_worked_df.rename(columns={'Time': 'Year', 'Value': 'Hours/Year'})
hours_worked_df

,Country,Year,Hours/Year
0,Australia,2000,1779.63
1,Australia,2001,1746.37
2,Australia,2002,1741.82
3,Australia,2003,1735.10
4,Australia,2004,1733.74
5,Australia,2005,1732.26
6,Australia,2006,1721.66
7,Australia,2007,1722.64
8,Australia,2008,1720.88
9,Australia,2009,1692.33


In [41]:
concat1 = pd.concat([hours_worked_df, unemployment_df], keys=['Country', 'Year'])
concat1

/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Country  Hours/Year  Unemployment (%)  Year
Country 0     Australia     1779.63               NaN  2000
        1     Australia     1746.37               NaN  2001
        2     Australia     1741.82               NaN  2002
        3     Australia     1735.10               NaN  2003
        4     Australia     1733.74               NaN  2004
        5     Australia     1732.26               NaN  2005
        6     Australia     1721.66               NaN  2006
        7     Australia     1722.64               NaN  2007
        8     Australia     1720.88               NaN  2008
        9     Australia     1692.33               NaN  2009
        10    Australia     1699.94               NaN  2010
        11    Australia     1699.92               NaN  2011
        12    Australia     1692.55               NaN  2012
        13    Australia     1689.38               NaN  2013
        14    Australia     1683.27               NaN  2014
        15    Australia     1683.55               NaN  2015
        16    Australia     1673.40               NaN  2016
        17    Australia     1674.90               NaN  2017
        18    Australia     1665.04               NaN  2018
        19      Austria     1542.00               NaN  2000
        20      Austria     1535.00               NaN  2001
        21      Austria     1530.00               NaN  2002
        22      Austria     1524.00               NaN  2003
        23      Austria     1530.00               NaN  2004
        24      Austria     1518.00               NaN  2005
        25      Austria     1505.00               NaN  2006
        26      Austria     1503.00               NaN  2007
        27      Austria     1506.00               NaN  2008
        28      Austria     1469.00               NaN  2009
        29      Austria     1469.00               NaN  2010
...                 ...         ...               ...   ...
Year    6261     Zambia         NaN             7.750  2015
        6262     Zambia         NaN             7.777  2016
        6263     Zambia         NaN             7.792  2017
        6264   Zimbabwe         NaN             4.913  1991
        6265   Zimbabwe         NaN             4.726  1992
        6266   Zimbabwe         NaN             4.844  1993
        6267   Zimbabwe         NaN             4.960  1994
        6268   Zimbabwe         NaN             5.452  1995
        6269   Zimbabwe         NaN             5.945  1996
        6270   Zimbabwe         NaN             6.930  1997
        6271   Zimbabwe         NaN             6.465  1998
        6272   Zimbabwe         NaN             6.000  1999
        6273   Zimbabwe         NaN             5.722  2000
        6274   Zimbabwe         NaN             5.496  2001
        6275   Zimbabwe         NaN             5.088  2002
        6276   Zimbabwe         NaN             4.726  2003
        6277   Zimbabwe         NaN             4.390  2004
        6278   Zimbabwe         NaN             4.506  2005
        6279   Zimbabwe         NaN             4.652  2006
        6280   Zimbabwe         NaN             4.757  2007
        6281   Zimbabwe         NaN             4.789  2008
        6282   Zimbabwe         NaN             5.112  2009
        6283   Zimbabwe         NaN             5.272  2010
        6284   Zimbabwe         NaN             5.370  2011
        6285   Zimbabwe         NaN             5.328  2012
        6286   Zimbabwe         NaN             5.205  2013
        6287   Zimbabwe         NaN             5.135  2014
        6288   Zimbabwe         NaN             5.086  2015
        6289   Zimbabwe         NaN             5.046  2016
        6290   Zimbabwe         NaN             5.011  2017

[7600 rows x 4 columns]

In [42]:
age_df

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,53.669919,54.056784,54.383281,54.710292,55.119933,55.631102,56.075544,56.703126,57.414449,...,69.161089,68.838092,68.882835,68.976476,69.090099,69.165181,69.159774,69.152468,69.003648,68.762184
1,Afghanistan,55.064659,54.747313,54.579029,54.500882,54.443084,54.376483,54.033502,53.738502,53.489183,...,49.815480,49.898603,50.319337,50.883182,51.545931,52.253100,52.974876,53.598917,54.174914,54.742297
2,Angola,52.719671,52.232669,51.635484,51.057092,50.703894,50.660431,50.358904,50.388691,50.631082,...,50.317153,50.370243,50.327156,50.349958,50.416077,50.505664,50.613384,50.690863,50.787584,50.917618
3,Albania,54.434567,54.068450,53.695003,53.353190,53.111451,53.010165,52.954356,53.062977,53.293096,...,66.619749,66.860680,67.520815,68.110597,68.645567,69.103098,69.436421,69.443689,69.415774,69.297902
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,53.171002,52.772366,52.421896,52.131270,51.923473,51.807219,51.575712,51.465944,51.441892,...,61.987849,62.260567,62.395755,62.452398,62.456183,62.451055,62.466139,62.436158,62.477525,62.556051
6,United Arab Emirates,51.880519,53.279645,54.808308,56.342481,57.804503,59.179181,60.504359,61.618012,62.489492,...,85.350921,85.872402,85.640650,85.525355,85.437623,85.322682,85.191607,85.028100,84.960562,84.903583
7,Argentina,63.695389,63.618909,63.572807,63.556931,63.570981,63.611345,63.596622,63.614682,63.650256,...,63.612670,63.721484,63.796777,63.827041,63.831450,63.843150,63.877761,63.880519,63.916238,63.973486
8,Armenia,54.997119,54.116450,53.462856,53.023164,52.767648,52.693961,52.808683,53.149812,53.686666,...,69.186079,69.545663,69.706505,69.722289,69.611188,69.436416,69.246199,69.015521,68.760873,68.490044
9,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
years_to_drop = [str(year) for year in range(1960,2000)]

age_df = age_df.drop(years_to_drop, axis=1)

In [44]:
age_df.loc[0].values

array(['Aruba', 69.24042133996679, 69.3233438825378, 69.4216355061479,
       69.5548202892277, 69.73474989112489, 69.9353200507843,
       69.8567914947636, 69.691760521636, 69.4483636399515,
       69.1610893714331, 68.838092240506, 68.8828353894545,
       68.9764762081168, 69.090098558927, 69.1651813671179,
       69.1597742018957, 69.1524679933602, 69.0036479708162,
       68.76218415822851], dtype=object)

In [45]:
age_years = pd.Series(age_df.columns[1:])
new_age_df = pd.DataFrame()
country_count = 0

for country in age_df.Country[:]:
    
    current_country_age = pd.Series(country)
    current_country_age = current_country_age.repeat(18).reset_index().loc[:,0]
    
    # adding years
    country_plus_years_age = pd.concat([current_country_age, age_years], axis=1)
    
    # value selection
    current_country_values_age = age_df.loc[country_count].values[1:]
    
    # putting it all together
    values_series_age = pd.concat([country_plus_years_age, pd.Series(current_country_values_age)], axis=1)
    
    new_age_df = pd.concat([new_age_df, values_series_age])
    new_age_df = new_age_df.reset_index(drop=True)
    
    country_count += 1
    
new_age_df.columns = ['Country', 'Year', 'Age']
new_age_df

,Country,Year,Age
0,Aruba,2000,69.2404
1,Aruba,2001,69.3233
2,Aruba,2002,69.4216
3,Aruba,2003,69.5548
4,Aruba,2004,69.7347
5,Aruba,2005,69.9353
6,Aruba,2006,69.8568
7,Aruba,2007,69.6918
8,Aruba,2008,69.4484
9,Aruba,2009,69.1611


In [59]:
new_income_df.head(50)

,0,0,0
0,Aruba,2001,17924.3
1,Aruba,2002,16894.5
2,Aruba,2003,18140.3
3,Aruba,2004,19503.3
4,Aruba,2005,16199.6
5,Aruba,2006,20567.5
6,Aruba,2007,19423.4
7,Aruba,2008,23229.6
8,Aruba,2009,21289.6
9,Aruba,2010,19741.1


In [47]:
new_age_df.head(5)

,Country,Year,Age
0,Aruba,2000,69.2404
1,Aruba,2001,69.3233
2,Aruba,2002,69.4216
3,Aruba,2003,69.5548
4,Aruba,2004,69.7347


In [48]:
edu_df.head(50)

,Entity,Year,Ave Years of Schooling
0,Afghanistan,1870,0.01
1,Afghanistan,1875,0.01
2,Afghanistan,1880,0.01
3,Afghanistan,1885,0.01
4,Afghanistan,1890,0.01
5,Afghanistan,1895,0.01
6,Afghanistan,1900,0.01
7,Afghanistan,1905,0.01
8,Afghanistan,1910,0.01
9,Afghanistan,1915,0.02


In [49]:
unemployment_df.head()

,Country,Year,Unemployment (%)
0,Afghanistan,1991,12.031
1,Afghanistan,1992,10.645
2,Afghanistan,1993,12.368
3,Afghanistan,1994,12.299
4,Afghanistan,1995,8.702


In [50]:
hours_worked_df.head()

,Country,Year,Hours/Year
0,Australia,2000,1779.63
1,Australia,2001,1746.37
2,Australia,2002,1741.82
3,Australia,2003,1735.10
4,Australia,2004,1733.74


In [60]:
new_income_df.to_csv("income.csv")

In [61]:
new_income_df

,0,0,0
0,Aruba,2001,17924.3
1,Aruba,2002,16894.5
2,Aruba,2003,18140.3
3,Aruba,2004,19503.3
4,Aruba,2005,16199.6
5,Aruba,2006,20567.5
6,Aruba,2007,19423.4
7,Aruba,2008,23229.6
8,Aruba,2009,21289.6
9,Aruba,2010,19741.1


In [66]:
new_age_df.to_csv("age_df.csv")

In [68]:
hours_worked_df.to_csv("hours_worked_df.csv")

In [69]:
edu_df.to_csv("edu_df.csv")

In [72]:
unemployment_df.to_csv("unemployment.csv")